In [1]:
# import csv into pandas dataframe
import pandas as pd

# read csv file
df = pd.read_csv('./subset_1_filtered_updated_final_output.csv')
print(df.info())


                                            pdf_path page_count  \
0  /home/ubuntu/west-filesys/indo-journal-pipelin...          8   
1  /home/ubuntu/west-filesys/indo-journal-pipelin...          7   
2  /home/ubuntu/west-filesys/indo-journal-pipelin...          4   
3  /home/ubuntu/west-filesys/indo-journal-pipelin...          4   
4                                 cutting speed  240     feed 0   

                                md_extraction_result  \
0  Edu Komputika 9 (2) (2022)\n\n![0_image_0.png]...   
1  \n\n![0_image_1.png](0_image_1.png)\n\nJURNAL ...   
2  \n\n![0_image_0.png](0_image_0.png)\n\n# Efekt...   
3  \n\n![0_image_0.png](0_image_0.png)\n\n![0_ima...   
4    Error: Failed to process PDF cutting speed  240   

                           extracted_meaningful_text  
0  Edu Komputika 9 ! Edu Komputika Journal ! http...  
1  ! JURNAL GEOGRAFI https://journal.unnes.ac.id/...  
2  ! # Efektivitas Penggunaan Modul Interaktif Te...  
3  ! ! Certicate of Publication is award

In [3]:
# drop rows in df where pdf_path is null and does not start with /home/ubuntu/ and does not contain "Error"
# Errors come from bad PDFs and errors from distributed pipeline
df_new = df[
    (df["pdf_path"].notna()) & 
    (df["pdf_path"].str.startswith("/home/ubuntu/")) & 
    (df["extracted_meaningful_text"].notna()) &  # Add this condition
    (~df["extracted_meaningful_text"].str.startswith("Error", na=False))
]

print(df_new.info())

<class 'pandas.core.frame.DataFrame'>
Index: 6018 entries, 0 to 6430
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   pdf_path                   6018 non-null   object
 1   page_count                 6018 non-null   object
 2   md_extraction_result       6018 non-null   object
 3   extracted_meaningful_text  6018 non-null   object
dtypes: object(4)
memory usage: 235.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 413 entries, 4 to 6405
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   pdf_path                   392 non-null    object
 1   page_count                 413 non-null    object
 2   md_extraction_result       413 non-null    object
 3   extracted_meaningful_text  413 non-null    object
dtypes: object(4)
memory usage: 16.1+ KB
None
5957                                   3 Sruwe

In [ ]:
# view all rows in df that are not in df_new, these are the rows that have errors
df_not_in_df_new = df[~df.index.isin(df_new.index)]

print(df_not_in_df_new.info())

print(df_not_in_df_new["pdf_path"].sample(10))

In [4]:
# Additional Checks that df_new does not contain error rows, should return empty
df_new[df_new["extracted_meaningful_text"].str.startswith("Error")]


,pdf_path,page_count,md_extraction_result,extracted_meaningful_text


In [6]:
# select only pdf_path, extracted_meaningful_text
df_new = df_new[["pdf_path", "md_extraction_result"]]

df_new.head()

,pdf_path,md_extraction_result
0,/home/ubuntu/west-filesys/indo-journal-pipelin...,Edu Komputika 9 (2) (2022)\n\n![0_image_0.png]...
1,/home/ubuntu/west-filesys/indo-journal-pipelin...,\n\n![0_image_1.png](0_image_1.png)\n\nJURNAL ...
2,/home/ubuntu/west-filesys/indo-journal-pipelin...,\n\n![0_image_0.png](0_image_0.png)\n\n# Efekt...
3,/home/ubuntu/west-filesys/indo-journal-pipelin...,\n\n![0_image_0.png](0_image_0.png)\n\n![0_ima...
6,/home/ubuntu/west-filesys/indo-journal-pipelin...,Edu Komputika 9 \n\n![0_image_0.png](0_image_0...


In [7]:
# view first row, for comparison with REGEX output below
print(df_new["md_extraction_result"].iloc[0])

Edu Komputika 9 (2) (2022)

![0_image_0.png](0_image_0.png)

Edu Komputika Journal

![0_image_2.png](0_image_2.png)

http://journal.unnes.ac.id/sju/index.php/edukom Pengembangan Game Math Story Problems Menggunakan RPG Maker MZ 

![0_image_1.png](0_image_1.png) untuk Anak 4-7 Tahun M. Nuraminudin1), Ika Asti Astuti 2)*, Sebastian Arby Susanto2) 1Jurusan Manajemen Informatika, Fakultas Ilmu Komputer, Universitas Amikom Yogyakarta, Indonesia 2Jurusan Sistem Informasi, Fakultas Ilmu Komputer, Universitas Amikom Yogyakarta, Indonesia

| Info Artikel ________________ Sejarah Artikel: Diterima: Juni 2022 Direvisi: Januari 2023 Disetujui: Januari 2023 ________________ Keywords: Game RPG, RPG Maker  MZ, Soal Cerita  Matematika _________________   | Abstrak   |
|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------|

Abstrak
__

In [9]:
import re
# now we can apply REGEX
def extract_meaningful_text(markdown_content):
    """Extract meaningful text from markdown content."""
    # Remove metadata and formatting
    content = re.sub(r'^---.*?---', '', markdown_content, flags=re.DOTALL)
    
    # Remove title tags but keep the title text
    content = re.sub(r'^#\s*(.*?)\n', r'\1\n', content)
    
    # Remove #### symbols but keep the header content
    content = re.sub(r'^####\s*(.*?)\n', r'\1\n', content, flags=re.MULTILINE)
    content = re.sub(r'^##\s*(.*?)\n', r'\1\n', content, flags=re.MULTILINE)
    
    content = re.sub(r'\*\*.*?\*\*', '', content)
    content = re.sub(r'\$.*?\$', '', content)
    
    # Remove citations and references
    content = re.sub(r'\[.*?\]', '', content)
    content = re.sub(r'\(.*?\)', '', content)
    
    # Remove figure captions and table content
    content = re.sub(r'Gambar \d+\..*', '', content)
    
    # Remove Markdown tables (improved version)
    content = re.sub(r'\|[^\n]*\|(\n\|[-:| ]+\|)?(\n\|[^\n]*\|)*', '', content)
    
    # Remove Keywords section
    content = re.sub(r'Keywords:.*?(?=\n\n)', '', content, flags=re.DOTALL)
    
    # Remove extra whitespace and newlines
    content = re.sub(r'\s+', ' ', content)
    content = content.strip()
    
    return content 

In [10]:
# apply extract_meaningful_text to md_extraction_result
df_new["regex_applied_text"] = df_new["md_extraction_result"].apply(extract_meaningful_text)

df_new.head()

,pdf_path,md_extraction_result,regex_applied_text
0,/home/ubuntu/west-filesys/indo-journal-pipelin...,Edu Komputika 9 (2) (2022)\n\n![0_image_0.png]...,Edu Komputika 9 ! Edu Komputika Journal ! http...
1,/home/ubuntu/west-filesys/indo-journal-pipelin...,\n\n![0_image_1.png](0_image_1.png)\n\nJURNAL ...,! JURNAL GEOGRAFI https://journal.unnes.ac.id/...
2,/home/ubuntu/west-filesys/indo-journal-pipelin...,\n\n![0_image_0.png](0_image_0.png)\n\n# Efekt...,! # Efektivitas Penggunaan Modul Interaktif Te...
3,/home/ubuntu/west-filesys/indo-journal-pipelin...,\n\n![0_image_0.png](0_image_0.png)\n\n![0_ima...,! ! Certicate of Publication is awarded to Din...
6,/home/ubuntu/west-filesys/indo-journal-pipelin...,Edu Komputika 9 \n\n![0_image_0.png](0_image_0...,Edu Komputika 9 ! # Edu Komputika Journal http...


In [12]:
print(df_new.iloc[0]["regex_applied_text"]) # compare this with above output to see the effect of REGEX

Edu Komputika 9 ! Edu Komputika Journal ! http://journal.unnes.ac.id/sju/index.php/edukom Pengembangan Game Math Story Problems Menggunakan RPG Maker MZ ! untuk Anak 4-7 Tahun M. Nuraminudin1), Ika Asti Astuti 2)*, Sebastian Arby Susanto2) 1Jurusan Manajemen Informatika, Fakultas Ilmu Komputer, Universitas Amikom Yogyakarta, Indonesia 2Jurusan Sistem Informasi, Fakultas Ilmu Komputer, Universitas Amikom Yogyakarta, Indonesia Abstrak _______________________________________________________________ Tujuan penelitian ini adalah memanfaatkan software RPG Maker MZ untuk membuat permainan RPG mengenai soal cerita matematika dan menguji kelayakannya sebagai alat bantu pembelajaran matematika. Metodologi penelitian dimulai dari analisis kebutuhan dan perencanaan dilanjutkan dengan implementasi serta validasi, review, dan pengujian produk dari calon pengguna maupun ahli. Spesifikasi fitur pada permainan serta perencangan desain interface dan karakter permainan dilakukan pada tahap analisis kebut